In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [3]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def get_mean_var(y_logit):
    y_logit=np.abs(y_logit-0.1)
    return np.mean(y_logit,axis=1)

def get_possibility(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=100, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
        y_logits=np.array(y_logits)
    return y_logits
import os
if not os.path.isdir('saved_models_{}'.format('dcgan')):
    os.mkdir('saved_models_{}'.format('dcgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('dcgan'), mode='w')
import torch.utils.data as Data
import cv2

from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from PIL import Image
import matplotlib.pyplot as plt

import sys
import os
import numpy as np

class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (X_test, _) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps = []
        values = []

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # labels = y_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise and generate a batch of new images
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator (real classified as ones and generated as zeros)
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator (wants discriminator to mistake images as real)
                g_loss = self.combined.train_on_batch(noise, valid)
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))


                if global_step % save_interval == 0:
                    self.mode_drop(epoch, global_step)
                    

    def mode_drop(self, epoch, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        # sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        # gen_imgs = 0.5 * gen_imgs + 0.5
        y_logits = get_possibility(gen_imgs)
        metrics = get_mean_var(y_logits)


        f.writelines('\n')
        f.writelines('global_step:' + str(global_step))
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in metrics)
        f.writelines('\n')
if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=50, batch_size=64, save_interval=200)


Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): BatchNorm2d(512, eps=1e-05

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 16, 16, 32)        896       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
batch_normalization_6 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 8, 8, 64)          0         
__________

epoch:0 step:56 [D loss: 0.186728, acc.: 94.53%] [G loss: 0.459123]
epoch:0 step:57 [D loss: 0.213630, acc.: 92.19%] [G loss: 0.678822]
epoch:0 step:58 [D loss: 0.176153, acc.: 96.09%] [G loss: 0.574987]
epoch:0 step:59 [D loss: 0.239362, acc.: 93.75%] [G loss: 0.608872]
epoch:0 step:60 [D loss: 0.397591, acc.: 81.25%] [G loss: 0.938434]
epoch:0 step:61 [D loss: 0.619071, acc.: 69.53%] [G loss: 1.242561]
epoch:0 step:62 [D loss: 0.811314, acc.: 57.81%] [G loss: 1.712152]
epoch:0 step:63 [D loss: 1.110198, acc.: 46.88%] [G loss: 1.895045]
epoch:0 step:64 [D loss: 1.048169, acc.: 52.34%] [G loss: 1.549279]
epoch:0 step:65 [D loss: 0.934177, acc.: 54.69%] [G loss: 1.022854]
epoch:0 step:66 [D loss: 0.667106, acc.: 69.53%] [G loss: 0.726846]
epoch:0 step:67 [D loss: 0.446826, acc.: 82.03%] [G loss: 0.611340]
epoch:0 step:68 [D loss: 0.332240, acc.: 86.72%] [G loss: 0.699833]
epoch:0 step:69 [D loss: 0.463681, acc.: 80.47%] [G loss: 0.770100]
epoch:0 step:70 [D loss: 0.504405, acc.: 77.34%]

epoch:0 step:180 [D loss: 0.459373, acc.: 75.00%] [G loss: 1.555543]
epoch:0 step:181 [D loss: 0.562998, acc.: 71.09%] [G loss: 1.189204]
epoch:0 step:182 [D loss: 0.519722, acc.: 75.00%] [G loss: 1.173047]
epoch:0 step:183 [D loss: 0.554624, acc.: 72.66%] [G loss: 0.867581]
epoch:0 step:184 [D loss: 0.498431, acc.: 74.22%] [G loss: 0.895129]
epoch:0 step:185 [D loss: 0.575146, acc.: 66.41%] [G loss: 0.945896]
epoch:0 step:186 [D loss: 0.652261, acc.: 60.16%] [G loss: 1.195312]
epoch:0 step:187 [D loss: 0.655370, acc.: 64.06%] [G loss: 1.019290]
epoch:0 step:188 [D loss: 0.761274, acc.: 53.12%] [G loss: 1.018791]
epoch:0 step:189 [D loss: 0.751081, acc.: 50.00%] [G loss: 1.410949]
epoch:0 step:190 [D loss: 0.602940, acc.: 64.06%] [G loss: 1.442718]
epoch:0 step:191 [D loss: 0.688492, acc.: 63.28%] [G loss: 1.699823]
epoch:0 step:192 [D loss: 0.561646, acc.: 70.31%] [G loss: 1.372007]
epoch:0 step:193 [D loss: 0.485091, acc.: 76.56%] [G loss: 1.128498]
epoch:0 step:194 [D loss: 0.608385

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:41: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

epoch:49 step:38493 [D loss: 0.662819, acc.: 60.16%] [G loss: 0.847551]
epoch:49 step:38494 [D loss: 0.654999, acc.: 64.06%] [G loss: 0.803167]
epoch:49 step:38495 [D loss: 0.647962, acc.: 58.59%] [G loss: 0.842927]
epoch:49 step:38496 [D loss: 0.653284, acc.: 56.25%] [G loss: 0.867637]
epoch:49 step:38497 [D loss: 0.669294, acc.: 57.81%] [G loss: 0.767002]
epoch:49 step:38498 [D loss: 0.704292, acc.: 51.56%] [G loss: 0.870259]
epoch:49 step:38499 [D loss: 0.671814, acc.: 60.94%] [G loss: 0.757873]
epoch:49 step:38500 [D loss: 0.653363, acc.: 59.38%] [G loss: 0.824414]
epoch:49 step:38501 [D loss: 0.693555, acc.: 52.34%] [G loss: 0.814218]
epoch:49 step:38502 [D loss: 0.687538, acc.: 48.44%] [G loss: 0.782080]
epoch:49 step:38503 [D loss: 0.677917, acc.: 59.38%] [G loss: 0.789251]
epoch:49 step:38504 [D loss: 0.686319, acc.: 45.31%] [G loss: 0.787591]
epoch:49 step:38505 [D loss: 0.655578, acc.: 61.72%] [G loss: 0.759894]
epoch:49 step:38506 [D loss: 0.661702, acc.: 62.50%] [G loss: 0.

epoch:49 step:38612 [D loss: 0.665636, acc.: 58.59%] [G loss: 0.731770]
epoch:49 step:38613 [D loss: 0.658682, acc.: 64.84%] [G loss: 0.818649]
epoch:49 step:38614 [D loss: 0.574202, acc.: 82.81%] [G loss: 0.871652]
epoch:49 step:38615 [D loss: 0.749854, acc.: 39.84%] [G loss: 0.718161]
epoch:49 step:38616 [D loss: 0.654669, acc.: 63.28%] [G loss: 0.774578]
epoch:49 step:38617 [D loss: 0.706486, acc.: 51.56%] [G loss: 0.927627]
epoch:49 step:38618 [D loss: 0.656552, acc.: 60.16%] [G loss: 0.822591]
epoch:49 step:38619 [D loss: 0.673927, acc.: 53.12%] [G loss: 0.782825]
epoch:49 step:38620 [D loss: 0.641640, acc.: 67.97%] [G loss: 0.903858]
epoch:49 step:38621 [D loss: 0.689270, acc.: 51.56%] [G loss: 0.861012]
epoch:49 step:38622 [D loss: 0.554374, acc.: 77.34%] [G loss: 0.875010]
epoch:49 step:38623 [D loss: 0.677173, acc.: 57.03%] [G loss: 0.797936]
epoch:49 step:38624 [D loss: 0.677899, acc.: 57.03%] [G loss: 0.781470]
epoch:49 step:38625 [D loss: 0.636378, acc.: 64.06%] [G loss: 0.

epoch:49 step:38730 [D loss: 0.644332, acc.: 60.94%] [G loss: 0.762947]
epoch:49 step:38731 [D loss: 0.677957, acc.: 57.03%] [G loss: 0.836508]
epoch:49 step:38732 [D loss: 0.658176, acc.: 57.81%] [G loss: 0.942433]
epoch:49 step:38733 [D loss: 0.643545, acc.: 67.19%] [G loss: 0.864327]
epoch:49 step:38734 [D loss: 0.653879, acc.: 67.19%] [G loss: 0.977155]
epoch:49 step:38735 [D loss: 0.610818, acc.: 71.09%] [G loss: 0.845646]
epoch:49 step:38736 [D loss: 0.634470, acc.: 68.75%] [G loss: 0.872694]
epoch:49 step:38737 [D loss: 0.693738, acc.: 57.81%] [G loss: 0.876287]
epoch:49 step:38738 [D loss: 0.685161, acc.: 53.91%] [G loss: 0.828178]
epoch:49 step:38739 [D loss: 0.671295, acc.: 63.28%] [G loss: 0.873466]
epoch:49 step:38740 [D loss: 0.616603, acc.: 67.19%] [G loss: 0.862315]
epoch:49 step:38741 [D loss: 0.716193, acc.: 51.56%] [G loss: 0.717585]
epoch:49 step:38742 [D loss: 0.790442, acc.: 35.16%] [G loss: 0.772085]
epoch:49 step:38743 [D loss: 0.686054, acc.: 57.81%] [G loss: 0.

epoch:49 step:38846 [D loss: 0.669747, acc.: 58.59%] [G loss: 0.718268]
epoch:49 step:38847 [D loss: 0.725824, acc.: 53.12%] [G loss: 0.755168]
epoch:49 step:38848 [D loss: 0.728088, acc.: 46.09%] [G loss: 0.787704]
epoch:49 step:38849 [D loss: 0.667380, acc.: 59.38%] [G loss: 0.812979]
epoch:49 step:38850 [D loss: 0.676259, acc.: 57.03%] [G loss: 0.727385]
epoch:49 step:38851 [D loss: 0.662524, acc.: 56.25%] [G loss: 0.889846]
epoch:49 step:38852 [D loss: 0.647420, acc.: 66.41%] [G loss: 0.724281]
epoch:49 step:38853 [D loss: 0.699800, acc.: 52.34%] [G loss: 0.762827]
epoch:49 step:38854 [D loss: 0.715671, acc.: 50.78%] [G loss: 0.775583]
epoch:49 step:38855 [D loss: 0.670076, acc.: 64.84%] [G loss: 0.770119]
epoch:49 step:38856 [D loss: 0.660226, acc.: 53.91%] [G loss: 0.856166]
epoch:49 step:38857 [D loss: 0.735791, acc.: 37.50%] [G loss: 0.596944]
epoch:49 step:38858 [D loss: 0.675175, acc.: 60.16%] [G loss: 0.817858]
epoch:49 step:38859 [D loss: 0.671845, acc.: 60.16%] [G loss: 0.

epoch:49 step:38962 [D loss: 0.687806, acc.: 52.34%] [G loss: 0.807060]
epoch:49 step:38963 [D loss: 0.657792, acc.: 64.84%] [G loss: 0.750479]
epoch:49 step:38964 [D loss: 0.682771, acc.: 60.16%] [G loss: 0.764958]
epoch:49 step:38965 [D loss: 0.718460, acc.: 49.22%] [G loss: 0.748587]
epoch:49 step:38966 [D loss: 0.670081, acc.: 59.38%] [G loss: 0.817547]
epoch:49 step:38967 [D loss: 0.598827, acc.: 70.31%] [G loss: 0.748736]
epoch:49 step:38968 [D loss: 0.702013, acc.: 46.88%] [G loss: 0.693338]
epoch:49 step:38969 [D loss: 0.646073, acc.: 64.06%] [G loss: 0.723900]
epoch:49 step:38970 [D loss: 0.727249, acc.: 42.97%] [G loss: 0.870928]
epoch:49 step:38971 [D loss: 0.705734, acc.: 53.91%] [G loss: 0.743513]
epoch:49 step:38972 [D loss: 0.712405, acc.: 47.66%] [G loss: 0.813561]
epoch:49 step:38973 [D loss: 0.675239, acc.: 60.94%] [G loss: 0.822988]
epoch:49 step:38974 [D loss: 0.762452, acc.: 41.41%] [G loss: 0.821248]
epoch:49 step:38975 [D loss: 0.637307, acc.: 64.06%] [G loss: 0.